```{contents}
```
## **Human Feedback in LangGraph**

**Human Feedback** in LangGraph is a **first-class control mechanism** that allows humans to **observe, intervene, correct, approve, and guide** an executing LLM workflow in real time.
It converts an LLM system from a **fully autonomous pipeline** into a **supervised, accountable decision system** suitable for production use.

---

### **1. Why Human Feedback Is Necessary**

LLM workflows involve uncertainty, risk, and evolving objectives.

| Problem             | Human Feedback Solves |
| ------------------- | --------------------- |
| Model hallucination | Manual correction     |
| Unsafe actions      | Approval gates        |
| Ambiguous goals     | Human clarification   |
| High-risk decisions | Accountability        |
| Long workflows      | Mid-course correction |

Without human feedback, complex LLM systems are **fragile and unsafe**.

---

### **2. Conceptual Model**

```
State → LLM Reasoning → Decision
                     ↓
               Human Review
                     ↓
        Approve / Edit / Reject / Redirect
                     ↓
               Updated State → Continue
```

Human feedback is **state mutation under supervision**.

---

### **3. Where Human Feedback Lives in LangGraph**

LangGraph implements feedback via:

* **Interrupt nodes**
* **Human nodes**
* **Approval gates**
* **State correction**
* **Conditional routing**

These mechanisms allow a human to **pause execution**, inspect state, modify it, and resume safely.

---

### **4. Basic Human Feedback Workflow**

### **State Definition**

```python
class State(TypedDict):
    task: str
    draft: str
    approved: bool
```

---

### **Human Review Node**

```python
def human_review(state):
    print("Draft:", state["draft"])
    user_input = input("Approve? (y/n): ")
    return {"approved": user_input == "y"}
```

---

### **Graph Construction**

```python
builder.add_node("generate", llm_generate)
builder.add_node("review", human_review)
builder.add_node("finalize", finalize_output)

builder.set_entry_point("generate")
builder.add_edge("generate", "review")

builder.add_conditional_edges(
    "review",
    lambda s: "finalize" if s["approved"] else "generate",
    {"finalize": "finalize", "generate": "generate"}
)
```

This produces a **controlled human-in-the-loop cycle**.

---

### **5. Types of Human Feedback in LangGraph**

| Type         | Purpose                 |
| ------------ | ----------------------- |
| Approval     | Allow or block progress |
| Correction   | Modify state content    |
| Redirection  | Change workflow path    |
| Interruption | Pause execution         |
| Supervision  | Continuous monitoring   |
| Override     | Replace model decision  |
| Annotation   | Add metadata            |
| Evaluation   | Score output quality    |

---

### **6. Production-Grade Human Feedback Patterns**

### **Approval Gate**

```
Model Decision → Human Approval → Execution Allowed
```

Used for: finance, healthcare, legal, infrastructure operations.

---

### **Correction Loop**

```
Draft → Human Edit → Model Revision → Human Check → Final
```

Used for: writing, code generation, compliance review.

---

### **Risk Escalation**

```
Low Risk → Auto Execute
High Risk → Human Review → Execute
```

---

### **7. Persistence & Auditing**

Human feedback integrates with:

* **Checkpointing**
* **State versioning**
* **Execution replay**
* **Audit logs**

This ensures:

| Property       | Benefit              |
| -------------- | -------------------- |
| Traceability   | Who changed what     |
| Accountability | Decision ownership   |
| Compliance     | Regulatory readiness |
| Recovery       | Safe resumption      |

---

### **8. Why This Matters for Agents**

Without human feedback, agents are:

* Non-auditable
* Unsafe in high-risk domains
* Hard to correct
* Untrustworthy in production

With human feedback, agents become:

> **Supervised decision systems**

---

### **9. Mental Model**

Human feedback in LangGraph is equivalent to:

> **Transactional control for AI decision making**

It ensures correctness, safety, and trust while preserving automation.


### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# ---------------------------
# 1. Define Shared State
# ---------------------------
class State(TypedDict):
    task: str
    draft: str
    approved: bool

# ---------------------------
# 2. LLM / Generator Node
# ---------------------------
def generate(state: State):
    draft = f"Draft answer for task: {state['task']}"
    return {"draft": draft}

# ---------------------------
# 3. Human Feedback Node
# ---------------------------
def human_review(state: State):
    print("\n--- HUMAN REVIEW ---")
    print("Task:", state["task"])
    print("Draft:", state["draft"])
    
    decision = input("Approve this draft? (y/n): ").strip().lower()
    
    if decision == "y":
        return {"approved": True}
    else:
        corrected = input("Enter corrected draft: ")
        return {"draft": corrected, "approved": False}

# ---------------------------
# 4. Finalization Node
# ---------------------------
def finalize(state: State):
    print("\nFINAL OUTPUT:", state["draft"])
    return state

# ---------------------------
# 5. Build Graph
# ---------------------------
builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("review", human_review)
builder.add_node("finalize", finalize)

builder.set_entry_point("generate")
builder.add_edge("generate", "review")

builder.add_conditional_edges(
    "review",
    lambda s: "finalize" if s["approved"] else "generate",
    {"finalize": "finalize", "generate": "generate"}
)

builder.add_edge("finalize", END)

graph = builder.compile()

# ---------------------------
# 6. Run
# ---------------------------
graph.invoke({"task": "Explain LangGraph human feedback", "approved": False})



--- HUMAN REVIEW ---
Task: Explain LangGraph human feedback
Draft: Draft answer for task: Explain LangGraph human feedback


Approve this draft? (y/n):  y



FINAL OUTPUT: Draft answer for task: Explain LangGraph human feedback


{'task': 'Explain LangGraph human feedback',
 'draft': 'Draft answer for task: Explain LangGraph human feedback',
 'approved': True}